In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
data

In [ ]:
data.area_type.value_counts()

In [ ]:
data = data.drop(['availability','balcony','society','area_type'],axis=1)
data

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()
data.isnull().sum()

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data['size'].unique()

In [ ]:
rooms=data['size'].str.split(expand=True)
data['bhk']=rooms[0];
data

In [ ]:
data.info()

In [ ]:
data.bhk = data.bhk.astype(int)
data

In [ ]:
data.info()

In [ ]:
data.bhk.unique()

In [ ]:
data[data.bhk>20]

In [ ]:
data.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
data[~data.total_sqft.apply(is_float)]

In [ ]:
def average(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
final_data=data.copy()
final_data.total_sqft=final_data.total_sqft.apply(average)
final_data = final_data[data.total_sqft.notnull()]
final_data

In [ ]:
final_data[final_data.total_sqft.isnull()]

Feature Engineering

In [ ]:
final_data

In [ ]:
final_data['price_per_sqft']=final_data['price']*100000/final_data['total_sqft']
final_data

In [ ]:
final_data.price_per_sqft.describe()

In [ ]:
len(final_data.location.unique())

In [ ]:
final_data.location = final_data.location.str.strip()
loc_stats=final_data.location.value_counts()
loc_stats

In [ ]:
len(loc_stats[loc_stats<=10])

In [ ]:
other = loc_stats[loc_stats<=10]
final_data.location = final_data.location.apply(lambda x: 'other' if x in other else x)
final_data.location.value_counts()

In [ ]:
len(final_data.location.unique())

Outlier Removal: 
We have kept our threshold to be 300 sqft per bhk

In [ ]:
final_data['sqft_per_bhk']=final_data.total_sqft/final_data.bhk
final_data=final_data[~(final_data.sqft_per_bhk<300)]
final_data

In [ ]:
final_data.price_per_sqft.describe()

In [ ]:
mean_l =final_data.groupby('location')['price_per_sqft'].mean()
mean_l

In [ ]:
std_l =final_data.groupby('location')['price_per_sqft'].std()
std_l

In [ ]:
diff = mean_l-std_l
diff=diff.to_frame()
diff.to_csv('diff.csv')

In [ ]:
diff = pd.read_csv('diff.csv')
diff

In [ ]:
final_data

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [ ]:
final_data = remove_pps_outliers(final_data)
final_data

In [ ]:
final_data.info()

In [ ]:
mean = final_data.groupby('bhk')['sqft_per_bhk'].mean()
mean

In [ ]:
plt.figure(figsize=(12,12))
sns.barplot(final_data.bhk,final_data.sqft_per_bhk)

In [ ]:
plt.figure(figsize=(12,12))
sns.histplot(final_data.price_per_sqft)

In [ ]:
final_data.bath.value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(final_data.bath)

In [ ]:
final_data[final_data.bath>10]

In [ ]:
final_data[final_data.bath>final_data.bhk+2]

In [ ]:
final_data = final_data[final_data.bath<final_data.bhk+2]
final_data

In [ ]:
data1 = final_data.drop(['size','price_per_sqft'],axis=1)
data2 = final_data.drop(['size','price_per_sqft','sqft_per_bhk'],axis=1)

In [ ]:
data1

In [ ]:
data2

In [ ]:
dum1 = pd.get_dummies(data1.location)
dum2 = pd.get_dummies(data2.location)
data1 = pd.concat([data1,dum1],axis=1)
data2=pd.concat([data2,dum2],axis=1)

In [ ]:
data1.shape,data2.shape

In [ ]:
data1

In [ ]:
X=final_data.drop(['location','price','size'],axis=1)
y=final_data.price

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor()
et.fit(X,y)

In [ ]:
et.feature_importances_

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=X.columns,y=et.feature_importances_)

In [ ]:
X = data1.drop(['price','location'],axis=1)
y = data1.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
X

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model_params = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
    'random_forest':{
        'model': RandomForestRegressor(),
        'params' : {
         'n_estimators':[int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)],
         'max_features':['auto','sqrt'],
         'max_depth':[int(x) for x in np.linspace(5,30, num = 6)],
        'min_samples_split':[2,5,10,15,100],
        'min_samples_leaf':[1,2,5,10]
        }
    }}

In [ ]:
scores =[]

for model_name, mp in model_params.items():
    clf=RandomizedSearchCV(mp['model'],mp['params'],cv=cv,return_train_score=False)
    clf.fit(X_train,y_train)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })

In [ ]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

In [ ]:
df.to_csv("params.csv")

In [ ]:
from sklearn import metrics
model = LinearRegression(normalize=False)
model.fit(X_train,y_train)
pred=model.predict(X_test)
score = metrics.r2_score(pred,y_test)
score

In [ ]:
X = data2.drop(['price','location'],axis=1)
y = data2.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train,y_train)

In [ ]:
xgb.score(X_train,y_train)

In [ ]:
pred=xgb.predict(X_test)
score = metrics.r2_score(pred,y_test)
score